This file compiles the county-level data.

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import geopandas as geopd
import us
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Which states
states = []
for state in us.states.STATES:
    states +=[state.abbr]
states += ['DC']

In [ ]:
# File path
root = ''
result_path = root + 'final_data/'
path_geodata = root + 'Data/geodata/'
path_data = root + 'Data/'

# Read data

In [ ]:
# Read in county shapefile
gdf_county = geopd.read_file(path_geodata + 'tl_2022_us_county/tl_2022_us_county.shp')
gdf_county = gdf_county[['GEOID','STATEFP','NAMELSAD','ALAND','geometry']]
gdf_county.rename(columns={'GEOID':'COUNTYFP','NAMELSAD':'COUNTYNAME','ALAND':'COUNTYALAND'},inplace=True)

# Read in population density in county

In [ ]:
# Population data in county
df_pop = pd.read_csv(path_data + 'US CENSUS/co-est2022-alldata.csv',index_col=[0])
df_pop['STATE'] = df_pop['STATE'].astype(str)
df_pop['COUNTY'] = df_pop['COUNTY'].astype(str)
df_pop['STATE'].loc[df_pop['STATE'].str.len() == 1] = '0' + df_pop['STATE'].loc[df_pop['STATE'].str.len() == 1]
df_pop['COUNTY'].loc[df_pop['COUNTY'].str.len() == 1] = '00' + df_pop['COUNTY'].loc[df_pop['COUNTY'].str.len() == 1]
df_pop['COUNTY'].loc[df_pop['COUNTY'].str.len() == 2] = '0' + df_pop['COUNTY'].loc[df_pop['COUNTY'].str.len() == 2]
df_pop['COUNTYFP'] = df_pop['STATE'] + df_pop['COUNTY']
df_pop.rename(columns={'POPESTIMATE2020':'Population_inCounty'},inplace=True)

In [ ]:
# Merge with gdf_county
gdf_county = gdf_county.merge(df_pop[['COUNTYFP','Population_inCounty']],how='left',left_on='COUNTYFP',right_on='COUNTYFP')
gdf_county['PopDensity_inCounty'] = gdf_county['Population_inCounty'] / gdf_county['COUNTYALAND']

# Cars

In [ ]:
# Read in residential car data (from CENSUS)
df_resCars_CENSUS_county = pd.read_csv(path_data + 'US_CENSUS/CENSUS_ResCars_bycounty.csv')
df_resCars_CENSUS_county['COUNTYFP'] = df_resCars_CENSUS_county['COUNTYFP'].astype(str)
df_resCars_CENSUS_county['COUNTYFP'].loc[df_resCars_CENSUS_county['COUNTYFP'].str.len() == 4] = '0' + df_resCars_CENSUS_county['COUNTYFP'].loc[df_resCars_CENSUS_county['COUNTYFP'].str.len() == 4]
df_resCars_CENSUS_county.head()

In [ ]:
# Assign actual county FIPS based on county shapefile
# Manually prepared file: Matching contains matching of BGs to new and old county FIPS in Connecticut
df_matching = pd.read_csv('CT_county_bg_mappings.csv',index_col=0)
df_matching = df_matching.drop_duplicates(subset=['COUNTYFP_BG','COUNTYFP_CT'])
df_matching = df_matching.dropna(subset=['COUNTYFP_CT'])
df_matching.head(3)

In [ ]:
# Compute full FIPS
df_matching['COUNTYFP_BG'] = df_matching['COUNTYFP_BG'].astype(str).str.zfill(3)
df_matching['COUNTYFP_CT'] = df_matching['COUNTYFP_CT'].astype(int).astype(str).str.zfill(3)

In [ ]:
# Merge
gdf_county = gdf_county.merge(df_resCars_CENSUS_county[['COUNTYFP','ResCars_inCounty']],how='left',left_on='COUNTYFP',right_on='COUNTYFP')
gdf_county.head(3)

In [ ]:
# Select columns
gdf_county.drop(['geometry'],axis=1,inplace=True)

# Save

In [ ]:
# Save
gdf_county.to_csv(result_path + 'level_county.csv')